In [1]:
from utils import *
from preprocessing import preprocess
from windowfy import windowfy
from featurizing import featurize
from tfidf_featurizer import combine_features, tfidf_featurize
from training import train, do_ensemble, do_train
from training_traditional import train_and_evaluate
from eval_erisk import evaluate, ensemble_vote
from IPython.display import display, Markdown
from itertools import product
from numpy.random import seed
import tensorflow
import numpy as np

/usr/local/lib/python3.6/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
seed(42)
tensorflow.random.set_seed(42) 
logger("Initialized numpy random and tensorflow random seed at 42")

Initialized numpy random and tensorflow random seed at 42


### With max_size = 10 and new data, sample_weights=10, oversample False, include_new_data=True

In [3]:
y_preds = {}

In [4]:
# prepare training data

In [7]:
train_users, y_train, test_users, y_test, train_samples, X_train, X_test = windowfy(window_size=10, max_size=100, sample_weights_size=100, is_oversample=True, include_new_data=False)
feats_train, feats_test = featurize(calculate_feats=True, include_feats=["first_prons","sentiment","nssi"], 
                       train_users=train_users, test_users=test_users, discretize=True, 
                                    discretize_size=50, dis_strategy="quantile", normalize=True)
tfidf_train, tfidf_test = tfidf_featurize(train_users, test_users, max_features=50000)

feats_train_comb, feats_test_comb = combine_features([tfidf_train, feats_train], [tfidf_test, feats_test])

feats_train_comb = feats_train_comb.toarray()
feats_test_comb = feats_test_comb.toarray()

Windowfying training users
[====================] 100%
Windowfying test users
[====================] 100%
Oversampling train users
After oversample: positive messages: 18744, negative messages: 23430
Data size: 42174

Finished windowfying
Featurizing calculate_feats=True, normalize=True, discretize=True, discretize_size=50, include_feats=['first_prons', 'sentiment', 'nssi']
Initialized numpy random and tensorflow random seed at 42
Data size: 42174, 42174
Data size: 25611, 25611
Calculating first prons
Calculating sentiment
Calculating NSSI words
Calculating first prons
Calculating sentiment
Calculating NSSI words
Normalizing features
Discretizing


/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 4 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:188: UserWarning: Feature 5 is constant and will be replaced with 0.
  "replaced with 0." % jj)
/home/ecampillo/.local/lib/python3.6/site-packages/sklearn/preprocessing/_discretization.py:222: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 6 are removed. Consider decreasing the number of bins.
  'decreasing the number of bins.' % jj)
/home/ecampillo/.loc

Is the combined the same from tfidf: False


In [8]:
# traditional classifiers

In [9]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="svm", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["svm"] = y_pred

              precision    recall  f1-score   support

           0       0.86      0.99      0.92     21304
           1       0.76      0.20      0.32      4307

    accuracy                           0.85     25611
   macro avg       0.81      0.59      0.62     25611
weighted avg       0.84      0.85      0.82     25611

[[21025   279]
 [ 3447   860]]
{'precision': 0.6388888888888888, 'recall': 0.6634615384615384, 'F1': 0.6509433962264151, 'ERDE_5': 0.26832387735217944, 'ERDE_50': 0.11832919319087853, 'median_latency_tps': 12.0, 'median_penalty_tps': 0.042873701496841665, 'speed': 0.9571262985031583, 'latency_weighted_f1': 0.6230350433652634}


In [10]:
y_pred, classifier = train_and_evaluate(feats_train_comb, y_train, feats_test_comb, y_test, train_samples, 
                                        classifier_name="bayes", strategy="weights")
eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
y_preds["bayes"] = y_pred

              precision    recall  f1-score   support

           0       0.90      0.90      0.90     21304
           1       0.52      0.53      0.53      4307

    accuracy                           0.84     25611
   macro avg       0.71      0.72      0.72     25611
weighted avg       0.84      0.84      0.84     25611

[[19237  2067]
 [ 2028  2279]]
{'precision': 0.3879310344827586, 'recall': 0.8653846153846154, 'F1': 0.5357142857142857, 'ERDE_5': 0.3280248286318115, 'ERDE_50': 0.11875810584759806, 'median_latency_tps': 11.0, 'median_penalty_tps': 0.03898023902249159, 'speed': 0.9610197609775084, 'latency_weighted_f1': 0.5148320148093795}


In [11]:
# deep learning classifiers
batch_size=32
model_name = "cnn_model"
iterations = 10

In [ ]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

y_preds[model_name] = model_resuls[max(model_resuls.keys())]

Starting training with cnn_model=cnn_model and maxlen=1000 and batch size=32
Generating embeddings
Data size: 42174
Training with callback
Evaluating
801/801 [==============================] - 8s 9ms/step - loss: 0.7590 - tp: 1073.0000 - fp: 321.0000 - tn: 20983.0000 - fn: 3234.0000 - accuracy: 0.8612 - precision: 0.7697 - recall: 0.2491 - f1_metric: 0.0888
Test Score: 0.759002149105072
Test Accuracy: 1073.0
801/801 [==============================] - 6s 7ms/step
Entered here
              precision    recall  f1-score   support

           0       0.87      0.98      0.92     21304
           1       0.77      0.25      0.38      4307

    accuracy                           0.86     25611
   macro avg       0.82      0.62      0.65     25611
weighted avg       0.85      0.86      0.83     25611

[[20983   321]
 [ 3234  1073]]
Finished training and evaluation
{'precision': 0.5147058823529411, 'recall': 0.6730769230769231, 'F1': 0.5833333333333334, 'ERDE_5': 0.284037982750046, 'ERDE_50':

In [ ]:
# deep learning classifiers
batch_size=32
model_name = "lstm_model_32"
iterations = 1

In [ ]:
model_resuls = {}
for i in range(0, iterations):
    y_pred = do_train(model_name=model_name, maxlen=1000, epochs=100, batch_size=batch_size,
                 shuffle=True, patience=30, feats_train=feats_train, feats_test=feats_test, 
                 X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test, train_sample_weights=train_samples)
    eval_resul = evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)
    model_resuls[eval_resul['latency_weighted_f1']] = y_pred

if len(model_resuls.keys() > 0):
    y_preds[model_name] = model_resuls[max(model_resuls.keys())]

In [ ]:
print(y_preds["cnn_model"].flatten())

In [ ]:
# ensemble

In [ ]:
y_pred = ensemble_vote(np.array([y_preds["svm"], y_preds["bayes"], y_preds["cnn_model"].flatten()]))
evaluate(1, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

In [ ]:
evaluate(2, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)

In [ ]:
evaluate(3, 10, {"test":"test"}, y_pred=y_pred, test_users=test_users)